# Liberally remove all games with imperfect or unhelpful data

In [176]:
import chess.pgn
import chess
import chess.engine
import math
import numpy
import pandas as pd
import os
import pyarrow
import logging
import re

# Configure logging
logging.basicConfig(filename='process.log', level=logging.INFO, 
                    format='%(asctime)s - %(message)s', datefmt='%H:%M:%S')

feather_path = '../lichess_db_standard_rated_2016-09.feather'
final_path = './lichess_2016-09_valid_games.feather'

engine_path = '/opt/homebrew/bin/stockfish'
engine_depth = 10

In [84]:
# Easily reusable funtion for future game imports
def delete_bad_games(delete_from_df):
    delete_bullet(delete_from_df)
    delete_correspondence(delete_from_df)
    delete_abandoned(delete_from_df)
    delete_unterminated(delete_from_df)
    delete_time_forfeit(delete_from_df)
    delete_rules_infraction(delete_from_df)
    

In [173]:
def delete_abandoned(delete_from_df):
    before = len(delete_from_df)
    abandoned_games_df = delete_from_df[delete_from_df['Termination'] == 'Abandoned']
    delete_from_df.drop(abandoned_games_df.index, inplace=True)
    return before - len(delete_from_df)

def delete_unterminated(delete_from_df):
    before = len(delete_from_df)
    unterminated_games_df = delete_from_df[delete_from_df['Termination'] == 'Unterminated']
    delete_from_df.drop(unterminated_games_df.index, inplace=True)
    return before - len(delete_from_df)

def delete_time_forfeit(delete_from_df):
    before = len(delete_from_df)
    time_forfeit_games_df = delete_from_df[delete_from_df['Termination'] == 'Time forfeit']
    delete_from_df.drop(time_forfeit_games_df.index, inplace=True)
    return before - len(delete_from_df)

def get_winner(row):
    if row['Result'] == '1-0':
        return 1
    elif row['Result'] == '0-1':
        return -1
    else:
        return 0

def get_loser_username(row):
    if row['Result'] == '1-0':
        return row['Black']  # Black lost
    elif row['Result'] == '0-1':
        return row['White']  # White lost
    return None  # Draw or other outcome
    
def delete_rules_infraction(delete_from_df):
    before = len(delete_from_df)
    rules_infraction_games_df = delete_from_df[delete_from_df['Termination'] == 'Rules infraction']

    cheaters = rules_infraction_games_df.apply(get_loser_username, axis=1)
    cheaters = cheaters[cheaters.notna()]
    cheater_list = cheaters.tolist()
    games_with_cheaters_df = delete_from_df[delete_from_df['White'].isin(cheater_list) | delete_from_df['Black'].isin(cheater_list)]
    
    delete_from_df.drop(games_with_cheaters_df.index, inplace=True)
    return before - len(delete_from_df)

def delete_bullet(delete_from_df):
    before = len(delete_from_df)
    bullet_games_df = delete_from_df[delete_from_df['Event'].str.find('Bullet') != -1]
    delete_from_df.drop(bullet_games_df.index, inplace=True)
    return before - len(delete_from_df)

def delete_blitz(delete_from_df):
    before = len(delete_from_df)
    blitz_games_df = delete_from_df[delete_from_df['Event'].str.find('Blitz') != -1]
    delete_from_df.drop(blitz_games_df.index, inplace=True)
    return before - len(delete_from_df)

def delete_classic(delete_from_df):
    before = len(delete_from_df)
    classic_games_df = delete_from_df[delete_from_df['Event'].str.find('Classic') != -1]
    delete_from_df.drop(classic_games_df.index, inplace=True)
    return before - len(delete_from_df)

def delete_correspondence(delete_from_df):
    before = len(delete_from_df)
    correspondence_games_df = delete_from_df[delete_from_df['Event'].str.find('Correspondence') != -1]
    delete_from_df.drop(correspondence_games_df.index, inplace=True)
    return before - len(delete_from_df)

def evaluate_board(eval_game, engine):
    moves = re.sub(r'\$\d+|\{[^}]*\}|1-0|0-1|1/2-1/2|\+\-', '', eval_game['Moves'])
    game = chess.pgn.Game()
    board = chess.Board()
    node = game

    moves = moves.split()

    for move in moves:
        if '.' not in move:
            board_move = board.parse_san(move)
            node = node.add_variation(board_move)
            board.push(board_move)

    return engine.analyse(board, chess.engine.Limit(depth=10))

def get_score(eval_info):
    return eval_info["score"].relative.score(mate_score=100000)

def get_predicted_leaver(subset_df, eval_threshold):
    eval_threshold = -abs(eval_threshold)
    leaver_games_idx = []
    logging.info(f"Starting delete predicted leaver for {len(subset_df)} games with threshold {eval_threshold}")
    with chess.engine.SimpleEngine.popen_uci(engine_path) as engine:
        for index, row in subset_df.iterrows():
            cur_info = evaluate_board(row, engine)
            cur_score = get_score(cur_info)
            if (cur_score > eval_threshold):
                leaver_games_idx.append(index)
    return leaver_games_idx


In [ ]:
df = pd.read_feather(feather_path) if os.path.exists(feather_path) else pd.DataFrame()
len(df)

### Consider termination type

In [5]:
df['Termination'].value_counts()

Termination
Normal              4635102
Time forfeit        2143022
Abandoned             34852
Rules infraction        132
Unterminated              5
Name: count, dtype: int64

#### Consider deleting abandoned games

In [6]:
abandoned_df = df[df['Termination'] == 'Abandoned']

In [7]:
abandoned_df.head()

,Event,Site,White,Black,Result,WhiteElo,BlackElo,WhiteRatingDiff,BlackRatingDiff,ECO,Opening,TimeControl,Termination,Moves
1,Rated Classical tournament https://lichess.org...,https://lichess.org/q3tbWY1W,Fherhuitron,Alequine,0-1,1346,1474,-25,+8,?,?,600+0,Abandoned,
22,Rated Classical tournament https://lichess.org...,https://lichess.org/I9FzSzig,Teagy,knightowl16,1-0,1955,1937,+10,-12,B00,King's Pawn,600+0,Abandoned,1. e4
26,Rated Blitz tournament https://lichess.org/tou...,https://lichess.org/XHWjOWL3,elgranmanny,Ded_Ahmed,0-1,1750,1756,-11,+12,?,?,180+0,Abandoned,
28,Rated Classical tournament https://lichess.org...,https://lichess.org/2mQSSU3s,Jorlu,fandeev,0-1,1927,1910,-13,+12,?,?,600+0,Abandoned,
32,Rated Blitz tournament https://lichess.org/tou...,https://lichess.org/PeJIqKDh,misadr2,salarabdi,1-0,1718,1727,+12,-17,B00,King's Pawn,300+0,Abandoned,1. e4


#### Delete abandoned games

In [8]:
delete_abandoned(df)

34852

#### Consider deleting unterminated games

In [9]:
unterminated_df = df[df['Termination'] == 'Unterminated']

In [10]:
unterminated_df[:3]

,Event,Site,White,Black,Result,WhiteElo,BlackElo,WhiteRatingDiff,BlackRatingDiff,ECO,Opening,TimeControl,Termination,Moves
518570,Rated Correspondence game,https://lichess.org/NuHYVTDl,Quasselstrippe,obelisk,*,1894,1937,None,None,A20,English Opening: King's English Variation,-,Unterminated,1. c4 e5 2. g3 d6 3. Bg2 g6 4. b3 f5 5. Bb2 Bg...
947053,Rated Correspondence game,https://lichess.org/Mkvwinr1,cunha18,carnefrita29,*,1311,1885,None,None,B20,Sicilian Defense: Staunton-Cochrane Variation,-,Unterminated,1. e4 c5 2. c4 Nc6 3. Nc3 e5 4. Nf3 d6 5. d3 g...
1605273,Rated Correspondence game,https://lichess.org/hUaPi7zA,Rikzz,bon-jovi,*,2002,1981,None,None,B06,Robatsch (Modern) Defense,-,Unterminated,1. e4 g6 2. d4 Bg7 3. c3 d6 4. Bc4 Nf6 5. Qc2 ...


In [11]:
unterminated_df['Event'].value_counts()

Event
Rated Correspondence game    5
Name: count, dtype: int64

#### Delete unterminated games

In [12]:
delete_unterminated(df)

5

#### Consider deleting rules infraction games

In [13]:
rules_infraction_df = df[df['Termination'] == 'Rules infraction']

In [14]:
rules_infraction_df[:3]

,Event,Site,White,Black,Result,WhiteElo,BlackElo,WhiteRatingDiff,BlackRatingDiff,ECO,Opening,TimeControl,Termination,Moves
21983,Rated Blitz game,https://lichess.org/HTFLtODp,naotomate,G4lvezLuciano,1-0,1621,1564,+9,-11,D51,"Queen's Gambit Declined: Modern, Knight Defens...",180+0,Rules infraction,1. d4 Nf6 2. c4 e6 3. Nc3 d5 4. Bg5 Nbd7 5. e3 h6
22156,Rated Blitz game,https://lichess.org/EtthZozB,GrofodHadzica,G4lvezLuciano,1-0,1658,1553,+11,-9,C42,Russian Game: Urusov Gambit,300+0,Rules infraction,1. e4 e5 2. Nf3 Nf6 3. Bc4 Nxe4 4. Nxe5 d5 5. ...
63994,Rated Classical game,https://lichess.org/y6vJXofx,bulychev691,TuxTux1,1-0,1771,1741,+9,-88,C84,Ruy Lopez: Closed Variations,600+10,Rules infraction,1. e4 e5 2. Nf3 Nc6 3. Bb5 a6 4. Ba4 Nf6 5. O-...


#### On the website it says cheating was detected for each game
#### Delete rules infraction games and games including the offending players

In [15]:
delete_rules_infraction(df)

8725

### Focus on blitz and classic/rapid
#### Delete bullet and correspondence games

In [35]:
bullet_df = df[df['Event'].str.find('Bullet') != -1]
blitz_df = df[df['Event'].str.find('Blitz') != -1]
classic_df = df[df['Event'].str.find('Classic') != -1]
correspondence_df = df[df['Event'].str.find('Correspondence') != -1]
print(len(bullet_df))
print(len(blitz_df))
print(len(classic_df))
print(len(correspondence_df))
print(len(bullet_df) + len(blitz_df) + len(classic_df) + len(correspondence_df))

1855459
3080600
1808263
25209
6769531


In [41]:
delete_bullet(df)

1855459

In [42]:
delete_correspondence(df)

25209

In [43]:
len(df)

4888863

#### Consider deleting games where players likely left

In [44]:
time_forfeit_df = df[df['Termination'] == 'Time forfeit']

In [151]:
time_forfeit_df[(time_forfeit_df['Moves'].str.find('7.') == -1) & (time_forfeit_df['Moves'].str.find('4.') != -1)].head()

,Event,Site,White,Black,Result,...,ECO,Opening,TimeControl,Termination,Moves
309,Rated Blitz game,https://lichess.org/lRe3VtJC,Flopcorn,eliel_123,1-0,...,B90,"Sicilian Defense: Najdorf Variation, Adams Attack",420+0,Time forfeit,1. e4 c5 2. Nf3 d6 3. d4 cxd4 4. Nxd4 Nf6 5. N...
563,Rated Classical game,https://lichess.org/Bh2z5c1N,Nico_Vera,TerahMet,1-0,...,C50,Giuoco Piano,1800+0,Time forfeit,1. e4 e5 2. Nf3 Nc6 3. Bc4 Bc5 4. Nc3 Nd4 5. N...
1285,Rated Blitz game,https://lichess.org/SfAEkx6Y,fou-roi,cetin78,1/2-1/2,...,A25,"English Opening: King's English Variation, Rev...",300+0,Time forfeit,1. c4 e5 2. Nc3 Nc6 3. g3 Bb4 4. Bg2 Na5 5. d3 d6
2144,Rated Blitz game,https://lichess.org/0XqRbB4K,Mwansa12472684,tanjawi1000,0-1,...,B00,Ware Defense,300+0,Time forfeit,1. e4 a5 2. Bc4 e6 3. Nf3 Nf6 4. e5 Ne4 5. d4 ...
2350,Rated Blitz game,https://lichess.org/jxtEACVU,portakal0001,tarkan01,0-1,...,C20,Bishop's Opening: Boi Variation,300+0,Time forfeit,1. e4 e5 2. Bc4 Bc5 3. Nf3 c6 4. Nxe5 b5


In [59]:
games_u6_df = time_forfeit_df[(time_forfeit_df['Moves'].str.find('6.') == -1)]
len(games_u6_df)

31249

#### Games where players timed out with less than 6 moves are very likely leaver games, we can just delete them

In [67]:
games_u10_6a_df = time_forfeit_df[(time_forfeit_df['Moves'].str.find('10.') == -1) & (time_forfeit_df['Moves'].str.find('6.') != -1)]
len(games_u10_6a_df)

24635

#### Games with less than 10 moves are likely as well, but now we must be cautious of someone letting their time run out because they are in a losing position
#### We must use an engine to see if the loser was in a losing position when they timed out

In [163]:
u10_6a_leavers_idx = get_predicted_leaver(games_u10_6a_df, -250)
len(u10_6a_leavers_idx)

17894

#### Compare % of timed out games from leaving at 6-9 moves vs 10-12 moves

In [158]:
games_u13_10a_df = time_forfeit_df[(time_forfeit_df['Moves'].str.find('13.') == -1) & (time_forfeit_df['Moves'].str.find('10.') != -1)]
len(games_u13_10a_df)

23626

In [162]:
u13_10a_leavers_idx = get_predicted_leaver(games_u13_10a_df, -250)
len(u13_10a_leavers_idx)

14361

#### Still a similar %, so we should go a little higher, up to the point where we can assume they were flagged vs leaving or rage quitting

In [160]:
games_u16_13a_df = time_forfeit_df[(time_forfeit_df['Moves'].str.find('16.') == -1) & (time_forfeit_df['Moves'].str.find('13.') != -1)]
len(games_u16_13a_df)

28346

In [164]:
u16_13a_leavers_idx = get_predicted_leaver(games_u13_10a_df, -250)
len(u16_13a_leavers_idx)

14361

#### These games continue to have high leaver rates, and time complexity for finding and deleting them is very high
#### Let's just delete all time forfeit games and get more data from the database if we need more games instead

In [174]:
delete_time_forfeit(df)

1144559

In [175]:
len(df)

3744304

In [177]:
df.to_feather(final_path)